In [111]:
import nltk
from nltk.collocations import *
from nltk.metrics.spearman import *
from scipy.stats import spearmanr
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models.phrases import Phraser, Phrases
import pandas as pd
import re

In [8]:
data = pd.read_csv('court-V-N.csv', header=None)
data.head()

,0,1,2,3,4
0,СУД,АРЕСТОВАТЬ,ПОДПОЛКОВНИК,ЗАМНАЧАЛЬНИК,ОТДЕЛ
1,СУД,ВЕСТИ,ПЕРЕГОВОРЫ,ПОСТАВКА,БУДУЩИЙ
2,СУД,ВЗЯТЬ,РЕАЛИЗАЦИЯ,ЗАКОН,ОБЯЗАТЕЛЬНЫЙ
3,СУД,ВЫКУПИТЬ,ГОСУДАРСТВО,АКЦИЯ,РАНЕЕ
4,СУД,ГОВОРИТЬ,ПРЕСС-СЕКРЕТАРЬ,ЕВГЕНИЙ,ВРЕМЕНИТЬ


In [29]:
rows = list(map(list, data.values))
tokens = [j for i in rows for j in i]

Выделим наиболее типичные для суда действия, применив 2 различных метода (оба являются вариациями алгоритма nltk bigrams, выбирают только те коллокации, которые встречаются больше 10 раз, но используют различные метрики):

#### NLTK Bigrams with PMI and Likelihood metrics

In [78]:
def freq_bigrams(tokens, metrics, nbest=10, freq_filter=0):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    metrics_={'pmi':bigram_measures.pmi, 'lr':bigram_measures.likelihood_ratio}
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(freq_filter)
    nbest = finder.nbest(metrics_[metrics], nbest)
    return nbest

In [96]:
bigrams_pmi = freq_bigrams(tokens, 'pmi', nbest=10, freq_filter=10)
bigrams_pmi = [''.join(i).lower().strip() for i in bigrams_pmi]
bigrams_pmi

['выдать санкция',
 'наложить арест',
 'санкция арест',
 'удовлетворить ходатайство',
 'признать виновная',
 'принять решение',
 'удовлетворить иск',
 'вынести решение',
 'суд рассмотреть',
 'суд дать']

In [97]:
bigrams_lr = freq_bigrams(tokens, 'lr', nbest=10, freq_filter=10)
bigrams_lr = [''.join(i).lower().strip() for i in bigrams_lr]
bigrams_lr

['суд суд',
 'суд признать',
 'выдать санкция',
 'суд удовлетворить',
 'удовлетворить иск',
 'удовлетворить ходатайство',
 'принять решение',
 'санкция арест',
 'наложить арест',
 'суд вынести']

In [108]:
import csv
for i, l in enumerate([bigrams_pmi, bigrams_lr]):
    with open("bigrams_{}.csv".format(['pmi', 'lr'][i]),'w') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        wr.writerows(l)

Пользуясь словарями, интуицией и т.п. составить золотой стандарт – ранжированный топ 10 коллокаций.

In [94]:
gold_standard = pd.read_csv('gold_standard.csv', header=None)

Сравним полученные результаты:

In [95]:
results = pd.concat([pd.Series(bigrams_pmi), pd.Series(bigrams_lr), gold_standard], axis=1)
results.columns = ['pmi', 'lr', 'gold']
results

,pmi,lr,gold
0,мера пресечение,суд суд,признать виновная
1,выдать санкция,суд признать,вынести приговор
2,отказать удовлетворение,выдать санкция,взять перерыв
3,наложить арест,суд удовлетворить,выслушать защита
4,санкция арест,удовлетворить иск,вызвать свидетель
5,санкционировать арест,удовлетворить ходатайство,принять решение
6,признать правота,принять решение,назначить наказание
7,удовлетворить ходатайство,санкция арест,вести переговоры
8,рассмотреть ходатайство,мера пресечение,освободить залог
9,признать виновная,наложить арест,удовлетворить ходатайство


In [118]:
print('Коэффициент корреляции Спирмана между золотым стандартом и nltk bigrams с метрикой pmi: ', 
      "%.2f" % spearmanr(results['pmi'], results['gold'])[0])

Коэффициент корреляции Спирмана между золотым стандартом и nltk bigrams с метрикой pmi:  -0.19


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [119]:
print('Коэффициент корреляции Спирмана между золотым стандартом и nltk bigrams с метрикой likelihood ratio: ', 
      "%.2f" % spearmanr(results['lr'], results['gold'])[0])

Коэффициент корреляции Спирмана между золотым стандартом и nltk bigrams с метрикой likelihood ratio:  0.12


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
